In [1]:
import os 
#import simpy
import random
import numpy as np
#import networkx as nx
#import matplotlib
#matplotlib.use('Gtk3Agg')
#import matplotlib.pyplot as plty
import collections
import re 
import string
import os 
import pickle

In [2]:
class G:
    # WORKING_DIRECTORY = "/Users/dghosal/Documents/HTM-Work"
    WORKING_DIRECTORY = os.getcwd()

    # Network paramaters
    number_of_columns = 1024
    cells_per_column = 16
    
    
    list_of_column_ids = list(range(1,number_of_columns+1))
    list_of_ids_in_a_column = list(range(1,cells_per_column+1))
    
    # cell_id = (colum_id, cell_in_column_id)
    
    
    # Cell parameters
    number_of_segments_min = 64
    number_of_segments_max = 64
    fixed_number_of_segments = 64
    
    
    # segment parameters
    max_num_synapses = 30
    min_num_synapses = 20
    fixed_num_synapses = 25
    
    theta = 16
    
     
    # synapse parameter
    
    max_p_value = 1
    min_p_value = 0

    alpha = 0.3 
    
    
    # synaptic strength update parameters 
    
    small_decrement = -0.05
    very_small_decrement = -0.01
    large_increment = 0.36
    
    
    # encoding parameters  
    encoding_length = 32  # number of columns activated for every input. 
    encoding_range = number_of_columns # 
    
    
    # This file will contain the encodings
    encodings_filename = "encodings_file.txt"
    
    
    
    # epoch length
    
    epoch_length = 12
    
    # training file
    pre_training_filename = "tkamb_training.txt"
    #training_filename = "tkamb_training.txt"
    #training_filename_long = "tkamb_training_long.txt"
    #training_filename = "tkamb.txt"
    
    #training_filename = "training_file-two-inputs.txt"

    training_filename = "mapped_data_22093.txt"
    
    # testing file
    testing_filename = "testing_file.txt"
    
    # output file
    output_filename = "output_file_v4_sm00_22093.txt"
    
    # pickle file to store the network
    pickle_filename = "my_network_v4_sm00.p"

In [3]:
class sparse_encoder_decoder(object):
    
    def __init__(self, encoding_length):
        
        self.encoding_length = encoding_length
        self.encoding_range = G.encoding_range
        self.encodings = {}  #his is a dictionary of the encodings 
        self.population = list(range(1, self.encoding_range +1))
    
    def sparse_encode(self, input_char):
        
        if input_char in self.encodings.keys():
            return(self.encodings[input_char])
        else:
            # give an input char it will generate an encoding and store the encodings 
            # the dictionary 
            # We will assume that we will prepopulate this dictionary
            # First generater sample of length K 
            
            temp = random.sample(self.population, self.encoding_length)
            e = set(sorted(temp))
        
            while e in self.encodings.values():
                temp = random.sample(self.population, self.encoding_length)
                e = set(sorted(temp))
            
            self.encodings.update({input_char:e})
            return(e)

    def sparse_decode(self, input_char):
        # given an input_char retiurs the tuple
        return(self.encodings[input_char])
    
  
    def load_current_encodings_from_dictionary(self, filename):
        file = open(filename, "r")
        lines = file.readlines()
        for line in lines: 
            c = line.split(":")
            key = c[0]
            value = eval(c[1])
            self.encodings.update({key:value})
        file.close()
        
    def store_current_encoding_in_dictionary(self,filename):
        file = open(filename, "w")
        for key in self.encodings.keys():
            line = str(key) + ":" + str(self.encodings[key]) + "\n"
            file.write(line)
        file.close()
        
    def generate_encodings(self, input_filename):
        
        # takes a filenname which is plain text 
        # removes all spaces and punctuations and get all the words and then the characters
        # generates the encodings
        inputfile = open(input_filename, 'r')
        lines = inputfile.readlines()
        for line in lines:
            words = re.findall(r'\w+', line) 
            #print(words)
            for word in words:
                for character in list(word):
                    self.sparse_encode(character.lower()) 
        
        inputfile.close()

        
   
    def print_encodings(self):
        
        # print the encoded dioctionary
        for key in self.encodings.keys():
            print(key, self.encodings[key])

In [4]:
class synapse(object):
    
    def __init__(self, synapse_id, segment_id, local_cell_id): 
        self.id = synapse_id                           # an integer
        self.segment_id = segment_id                   # which segment does the synapse belong to
        self.local_cell_id = local_cell_id                   # which cell does it belong to. This is a tuple
        self.remote_cell_id = ()                       # which cell does it connect to 
                                                       #   we assume each synapse connect to a single cell which is a tuple
        self.p_value = np.random.uniform(0,1)          # the permanance value which is initialized to a value between 0 and 1
        
    
    def get_p_value(self):
        return(self.p_value)
    
    def update_p_value(self, p_value_change):
        old_p_value = self.p_value
        self.p_value = min(max(self.p_value + p_value_change,0),1)
        #print("Updated synapse {0} in segment {1} in cell {2}  with remote cell_id {3} with current p_Value {4} updated to new p_value {5}".format(self.id, self.segment_id, self.local_cell_id, self.remote_cell_id, old_p_value, self.p_value))
        
    
    def get_id(self): 
        return(self.id)
        
    def get_segment_id(self):
        return(self.segment_id)
    
    def get_local_cell_id(self):                       # which cell does this synapse belong to
        return(self.cell_id)
    
    def assign_remote_cell_id(self, remote_cell_id):
        self.remote_cell_id = remote_cell_id
    
    def get_remote_cell_id(self):             # Which cell does this connect to 
        return(self.remote_cell_id)

In [5]:
class segment(object):
    
    def __init__(self, segment_id, cell_id, num_synapses):
        self.segment_id  = segment_id                # an integer id
        self.local_cell_id = cell_id               # which cell does this segmeent belong to
        self.num_synapses = num_synapses     # number of synapses
        self.segment_synapses = {}           # dictionary of synapses of this segment. key is the synapse id  value is the synapse
        self.dij = {}                        # dictionary key synapseid value is the permanence value
        self.remote_cell_ids = set()            # a set containing all the remote cell ids of the segment
        self.tildedij = {}                    # dictionary key synapseid value remote-cell-id for synapses with p_value > G.alpha
        self.tildedij_set = set()                 # This is just the set consisting of the remote cellids of connected synapse
        self.dotdij = {}                     # dictionary key synapseid value remote-cell-id for synapses with p_value > 0
        self.dotdij_set = set()                  # This is just the set consisting of the remote cellids of synapses with +ve p_value
    
    def create_synapses(self): 
        
        # this function will create the synapses 
        # initilize dij, tildedij, and dotdij the dictionaries and the sets
                
        for i in list(range(1,self.num_synapses+1)):
            
            # create the synapse
            s = synapse(i, self.segment_id, self.local_cell_id)
            
            # store the synapse
            self.segment_synapses[i] = s
            
            # connect the synapse to a remote cell
            # generate a random cell_id 
            # we want to makem sure that all the synapses of the segment connect to 
            #   distinct cells 

            x = np.random.randint(1, G.number_of_columns+1)
            y = np.random.randint(1, G.cells_per_column+1)
            remote_cell_id = (x,y)
            while (remote_cell_id in self.remote_cell_ids) or (remote_cell_id == self.local_cell_id): 
                x = np.random.randint(1, G.number_of_columns+1)
                y = np.random.randint(1, G.cells_per_column+1)
                remote_cell_id = (x,y)
            
            #print(remote_cell_id)
            # assign the remoote cell-id to the synapse
            s.assign_remote_cell_id(remote_cell_id)
            
            p_value = s.get_p_value()
            
            # add it to dij 
            #k = str(i) + ':' + str(remote_cell_id)
            self.dij.update({i:p_value})
            
            # add the remote cell_id to the set self.remote_cell_ids
            self.remote_cell_ids.add(remote_cell_id) 
            
            # add to dotdij 
            if p_value > 0:
                self.dotdij.update({i:remote_cell_id})
                self.dotdij_set.add(remote_cell_id)

            
            # add to tildedij 
            if p_value > G.alpha:
                self.tildedij.update({i:remote_cell_id})
                self.tildedij_set.add(remote_cell_id)
 

In [6]:
class cell(object):
    
    def __init__(self, cell_id, num_segments):
        self.cell_id = cell_id                 # this is a tuple
        self.num_segments = num_segments       # how many segments does the cell have
        self.cell_segments = {}                # dictionary of all the segments in the cell
                                               #   the segment-id is the key and segment object is the value 

        self.activity_state = 0
        self.predictive_state = 0
        
        self.active_segments = []             # this is a set of segment ids that have been found to be active                                   # 
        self.segment_strengths = {}                                # create dictionary segment_id: strength wrt to at

        
    def create_segments(self):
        
        for i in list(range(1, self.num_segments+1)):
            #num_synapses = np.random.randint(G.min_num_synapses, G.max_num_synapses)   
            num_synapses = G.fixed_num_synapses  
            s = segment(i, self.cell_id, num_synapses)
            s.create_synapses()
            self.cell_segments[i] = s

            
            
    def get_activity_state(self):
        
        return(self.activity_state)
        
        
    def get_predictive_state(self):
        
        return(self.predictive_state)
        
        
    def update_activity_state(self, value):
        
        # recompute the activity state of the cell   
        
        self.activity_state = value
              

    def compute_predictive_state(self, at):
        
        # this function will compute the predictive state given the currrent feedforward input 
        # We will break as soon as we find the first active segment
        
        count = 0
        
        for segment_id in list(range(1, self.num_segments+1)):
            
            # for this segment find the overlapp with at 
            overlapp = self.cell_segments[segment_id].compute_overlapp_with_tidledij(at)

            if len(overlapp) > G.theta:
                count += 1
                break     
            
        if count == 0:
            self.predictive_state = 0
            return(0)
        else:
            self.predictive_state = 1
            return(1)
    
    
    
    def determine_segment_id_causing_predictive_state(self, at):
        
        # given a set of cell ids in at
        # generates a dictionary of  segment_id and strengths that caused this cell to be in the predictive state  
        
        self.segment_ids_causing_predictive_state = {}
        for segment_id in self.cell_segments.keys(): 
            overlapp = self.cell_segments[segment_id].tildedij_set.intersection(at)
            if len(overlapp) > G.theta:
                self.segment_ids_causing_predictive_state.update({segment_id:len(overlapp)})
        #print("Segments causing predictive state: {0}".format(self.segment_ids_causing_predictive_state))
        

    def determine_segment_id_strengths(self, at):
        
        # given a set of cell ids in at
        # returns a dictionaries of  segment_id and their overlapps
        
        self.segment_id_strengths = {}
        
        for segment_id in self.cell_segments.keys(): 
            overlapp = self.cell_segments[segment_id].dotdij_set.intersection(at)
            self.segment_id_strengths.update({segment_id:overlapp})